In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# ! pip install gym

In [3]:
from gym.spaces import Discrete
from gym.spaces import Tuple
from gym.spaces.box import Box

In [4]:
throttle_brake = Discrete(3)  # -1 brake, 0 keep, 1 throttle
steering = Discrete(3)
action_space = Tuple([throttle_brake,steering])

N = 4 # FIXME not hard code
F = 6 # FIXME not hard code
state_space = Box(low=-np.inf, high=np.inf, shape=(N,F), dtype=np.float32)

In [6]:
from tensorflow.keras.layers import Input, Dense, Lambda, Multiply, Reshape, Flatten, Masking, LSTM, Concatenate
from tensorflow.keras.models import Model



In [14]:
CAV_state_input = Input(name="cav_input",shape=(6,))
CAV_action_input = Input(name="cav_action",shape=(3,)) #throttle, brake, steering
input_concate = Concatenate(axis=-1)([CAV_state_input,CAV_action_input])
x = Dense(32,activation='relu',name="CAV_dynamics_1")(input_concate)
x = Dense(16,activation='relu',name="CAV_dynamics_2")(x)
out = Dense(6,activation=None)(x)
CAV_dynamics = Model(inputs=[CAV_state_input,CAV_action_input], outputs=out)
CAV_dynamics.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cav_input (InputLayer)          [(None, 6)]          0                                            
__________________________________________________________________________________________________
cav_action (InputLayer)         [(None, 3)]          0                                            
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 9)            0           cav_input[0][0]                  
                                                                 cav_action[0][0]                 
__________________________________________________________________________________________________
CAV_dynamics_1 (Dense)          (None, 32)           320         concatenate_7[0][0]        

In [17]:
HDV_state_input = Input(name="hdv_input",shape=(6,))
x = Dense(32,activation='relu',name="HDV_dynamics_1")(HDV_state_input)
x = Dense(16,activation='relu',name="HDV_dynamics_2")(x)
out = Dense(6,activation=None)(x)
HDV_dynamics = Model(inputs=HDV_state_input, outputs=out)
HDV_dynamics.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hdv_input (InputLayer)       [(None, 6)]               0         
_________________________________________________________________
HDV_dynamics_1 (Dense)       (None, 32)                224       
_________________________________________________________________
HDV_dynamics_2 (Dense)       (None, 16)                528       
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 102       
Total params: 854
Trainable params: 854
Non-trainable params: 0
_________________________________________________________________


# Reverse the control commands

In [7]:
import glob
import pickle

In [10]:
!pwd

/Users/jiqiandong/Desktop/OneDrive - purdue.edu/work2/carla_rl


In [12]:
with open("./control_details/LHDV.p", 'rb') as f:
    commands = pickle.load(f)

In [16]:
commands_reversed = []
for command in commands:
    c = command.copy()
    c['steer'] = -command['steer']
    commands_reversed.append(c)

In [17]:
commands_reversed[50]

{'throttle': 0.0,
 'steer': -0.0,
 'brake': 0.0,
 'hand_brake': 0,
 'reverse': False,
 'manual_gear_shift': True,
 'gear': 0}

In [18]:
commands[50]

{'throttle': 0.0,
 'steer': 0.0,
 'brake': 0.0,
 'hand_brake': 0,
 'reverse': False,
 'manual_gear_shift': True,
 'gear': 0}

In [19]:
import random

In [31]:
random.choice([0,1])

0

In [32]:
with open("./control_details/LHDV_right.p", 'wb') as f:
    pickle.dump(commands_reversed,f)

In [1]:
import numpy as np

In [2]:
a = np.array([[1,2,3],[4,5,6]])
b = a*2

In [3]:
a

array([[1, 2, 3],
       [4, 5, 6]])

In [4]:
b

array([[ 2,  4,  6],
       [ 8, 10, 12]])

In [7]:
np.mean([a,b],axis=0)

array([[1.5, 3. , 4.5],
       [6. , 7.5, 9. ]])

In [1]:
a = {'1':1,'b':2}

In [3]:
for key,val in a.items():
    print(key,val)

1 1
b 2


In [6]:
for item in a:
    print(a[item])

1
2


In [6]:
a = [[[1,2,3],[4,5,6]],[[1,2,3],[4,5,6]]]

In [7]:
a

[[[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]]

In [10]:
import numpy as np
np.array(a)[-1,:,:]

array([[1, 2, 3],
       [4, 5, 6]])

## Test dataset

In [1]:
import pickle
with open('./experience_data/data_pickle.pickle','rb') as f:
    d = pickle.load(f)

In [2]:
import torch
from collections import defaultdict

In [3]:
def random_iterator(self, batch_size, return_sequence=True):
    """
    Iterate once through all (s, a, r, s') in batches in a random order
    For only training the system dynamic function only.
    """
    all_indices = np.nonzero(np.logical_not(self._dones))[0]
    np.random.shuffle(all_indices)
    actions = np.array(self._actions)

    i = 0
    while i < len(all_indices):
        indices = all_indices[i:i+batch_size]
        output_state = defaultdict(list)
        output_next_state = defaultdict(list)
        for ind in indices:
            current = self._states[ind]
            next_ = self._next_states[ind]
            for key, val in current.items():
                output_state[key].append(val)

            for key,val in next_.items():
                if return_sequence:
                    output_next_state[key].append(val)
                else:
                    output_next_state[key].append(val[-1,:])

        for key in output_state.keys():
            output_state[key] = torch.tensor(output_state[key]).float()

        for key in output_next_state.keys():
            output_next_state[key] = torch.tensor(output_next_state[key]).float()
            
        yield output_state, actions[indices], output_next_state

        i += batch_size

In [11]:
torch.tensor(next(d.random_iterator(3))[1]).float().shape

torch.Size([3, 2])

In [12]:
next(d.random_iterator(0))

(defaultdict(list, {}),
 array([], shape=(0, 2), dtype=int64),
 defaultdict(list, {}))